# Часть 1: Сбор данных

## Описание данных:
На этапе сбора данных была реализована автоматизированная выгрузка вакансий с платформы hh.ru по релевантным ключевым словам, связанным с направлением подготовки "Бизнес-информатика" университета ИТМО.
Сбор осуществлялся с использованием API hh.ru, с применением фильтрации по географическому признаку (Санкт-Петербург) и ключевым навыкам, востребованным на рынке труда (например, "Бизнес-аналитик", "Product Manager", "SQL", "Python" и др.).


---


Полученный датасет включает сведения о названии вакансии, компании, формате и графике работы, требованиях к опыту, а также предложениях по заработной плате. Также осуществлялась попытка получения дополнительной информации о работодателе (например, рейтинга, если доступен).


---


Сбор данных был завершён 29.04.2025.

## Импорты и настройки логирования

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm
import time
import logging
from datetime import datetime

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='hh_parser.log'
)
logger = logging.getLogger(__name__)


## Константы и базовые параметры

In [ ]:
BASE_URL = 'https://api.hh.ru/vacancies'
EMPLOYER_URL = 'https://api.hh.ru/employers/'
HEADERS = {
    'User-Agent': 'MyAwesomeBot/1.0'
}


## Универсальная функция запроса fetch

In [ ]:
def fetch(url, params=None, retries=3, delay=1.0):
    """
    Выполняет HTTP GET-запрос с повторными попытками при ошибках.

    :param url: URL запроса
    :param params: словарь параметров запроса
    :param retries: количество повторных попыток
    :param delay: задержка между попытками
    :return: JSON-ответ или None
    """
    for attempt in range(retries):
        try:
            response = requests.get(url, params=params, headers=HEADERS)
            if response.status_code == 200:
                return response.json()
            elif 400 <= response.status_code < 500:
                logger.error(f"Клиентская ошибка {response.status_code} на {url}: {response.text}")
                return None
            elif 500 <= response.status_code < 600:
                logger.warning(f"Серверная ошибка {response.status_code} на {url}, попытка {attempt + 1}")
                time.sleep(delay * (2 ** attempt))
        except requests.RequestException as e:
            logger.error(f"Ошибка запроса: {e}, попытка {attempt + 1}")
            time.sleep(delay * (2 ** attempt))
    logger.error(f"Не удалось получить данные после {retries} попыток: {url}")
    return None


## Получение рейтинга компании

In [ ]:
def fetch_employer_rating(employer_id):
    """
    Получает рейтинг компании по ID работодателя.

    :param employer_id: ID работодателя
    :return: строка с рейтингом или 'Нет рейтинга'
    """
    if not employer_id:
        return 'Нет рейтинга'
    data = fetch(f"{EMPLOYER_URL}{employer_id}")
    if data:
        return data.get('rating', 'Нет рейтинга')
    return 'Нет рейтинга'


## Обработка одной вакансии

In [ ]:
def process_vacancy(vac):
    """
    Обрабатывает отдельную вакансию в удобный для анализа формат.

    :param vac: словарь с вакансией
    :return: словарь с обработанными данными
    """
    salary = vac.get('salary')
    salary_str = 'Не указано'
    if salary:
        salary_from = salary.get('from')
        salary_to = salary.get('to')
        currency = salary.get('currency', 'RUR')
        if salary_from and salary_to:
            salary_str = f"{salary_from} - {salary_to} {currency}"
        elif salary_from:
            salary_str = f"от {salary_from} {currency}"
        elif salary_to:
            salary_str = f"до {salary_to} {currency}"

    experience = vac.get('experience', {}).get('name', 'Не указано')
    employment = vac.get('employment', {}).get('name', 'Не указано')
    schedule = vac.get('schedule', {}).get('name', 'Не указано')
    address = vac.get('address', {})
    work_format = 'Удаленная работа' if vac.get('remote_work') else 'Офис' if address else 'Не указано'

    employer_id = vac.get('employer', {}).get('id')
    rating = fetch_employer_rating(employer_id)

    return {
        'Название вакансии': vac.get('name', 'Не указано'),
        'Компания': vac.get('employer', {}).get('name', 'Не указано'),
        'Рейтинг компании': rating,
        'Зарплата': salary_str,
        'Опыт работы': experience,
        'Тип занятости': employment,
        'График работы': schedule,
        'Рабочие часы': 'Не указано',
        'Формат работы': work_format,
        'Ссылка на вакансию': vac.get('alternate_url', 'Не указано')
    }


## Поиск и обработка вакансий по ключевому слову

In [ ]:
def fetch_vacancies(keyword, area=2):
    """
    Получает и обрабатывает вакансии для одного ключевого слова.

    :param keyword: ключевое слово для поиска
    :param area: ID региона (по умолчанию 2 — Санкт-Петербург)
    :return: список обработанных вакансий
    """
    logger.info(f"Поиск вакансий для: {keyword}")
    page = 0
    raw_vacancies = []
    seen_ids = set()

    while page < 100:
        params = {
            'text': keyword,
            'area': area,
            'page': page,
            'per_page': 20
        }
        data = fetch(BASE_URL, params)
        if not data or 'items' not in data or not data['items']:
            break
        items = data['items']
        raw_vacancies.extend(items)
        page += 1
        time.sleep(1.5)

    logger.info(f"Найдено {len(raw_vacancies)} вакансий для '{keyword}'")

    processed = []
    for vac in tqdm(raw_vacancies, desc=f"Обработка вакансий ({keyword})"):
        url = vac.get('alternate_url')
        vac_id = url.split('/')[-1] if url else None
        if vac_id and vac_id not in seen_ids:
            seen_ids.add(vac_id)
            processed.append(process_vacancy(vac))

    logger.info(f"Обработано {len(processed)} уникальных вакансий для '{keyword}'")
    return processed


## Основная функция запуска main()

In [ ]:
def main():
    """
    Основная функция парсера: собирает вакансии по всем ключевым словам.
    """
    keywords = [
        'Бизнес-аналитик', 'Системный аналитик', 'Аналитик данных', 'Product Manager',
        'Project Manager', 'IT-консультант', 'Корпоративные информационные системы',
        'Проектное управление', 'Тестировщик', 'Business Analyst', 'System Analyst',
        'Data Analyst', 'QA Engineer', 'Продуктовый менеджер', 'Менеджер проектов',
        'SQL', 'BPMN', 'UML', 'Agile', 'Scrum', 'Jira', 'Confluence', 'Python',
        'Tableau', 'Power BI', 'API', 'REST', 'SOAP', 'SAP', '1С', 'ERP', 'CRM',
        'Automation Testing', 'Selenium', 'A/B-тестирование', 'Roadmap', 'MVP',
        'UX/UI', 'PMP', 'Kanban', 'AWS', 'Azure', 'BI-аналитик', 'Финтех', 'E-commerce'
    ]

    all_vacancies = []
    for keyword in keywords:
        vacancies = fetch_vacancies(keyword)
        all_vacancies.extend(vacancies)
        time.sleep(2.0)

    logger.info(f"Всего вакансий собрано: {len(all_vacancies)}")

    df = pd.DataFrame(all_vacancies)
    output_file = f'vacancies_spb_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
    df.to_excel(output_file, index=False)
    logger.info(f"Данные сохранены в файл: {output_file}")


## Запуск скрипта

In [ ]:
if __name__ == "__main__":
    logger.info("Запуск парсера hh.ru")
    main()


Обработка вакансий (Бизнес-аналитик): 100%|██████████| 316/316 [02:48<00:00,  1.88it/s]
